# Credit Card Fraud Detection
https://www.kaggle.com/mlg-ulb/creditcardfraud

### Context
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

### Content
The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [1]:
import pandas as pd
from sklearn.ensemble import *
from sklearn.tree import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import *
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
from sklearn.svm import *
from sklearn.neighbors import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.gaussian_process import *

In [2]:
df = pd.read_csv('creditcard.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Average Fraud Amount

In [4]:
df['Amount'].groupby(df['Class']).mean()

Class
0     88.291022
1    122.211321
Name: Amount, dtype: float64

Average Fraud Amount is 122.2

In [5]:
x = df.drop(['Class'],axis=1)
y = df['Class']

### Splitting the Dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)

In [7]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf",probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    RidgeClassifierCV()]

Recall = TruePositives / (TruePositives + FalseNegatives)

Precision = TruePositives / (TruePositives + FalsePositives)

F1 = 2 (precision recall) / (precision + recall)

In [8]:
def f_score(X_train, X_test, y_train, y_test):
    for clf in classifiers:
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        f = f1_score(y_true=y_test,y_pred=y_pred,average="macro")
        roc = roc_auc_score(y_true=y_test,y_score=y_pred,average="macro")
        print(f,roc,clf.__class__.__name__)

In [9]:
f_score(X_train, X_test, y_train, y_test)

0.5860273113962261 0.5472972972972973 KNeighborsClassifier
0.49956659247979385 0.5 SVC
0.8731832164419524 0.8983555516687199 DecisionTreeClassifier
0.9327303232985232 0.8918684441334976 RandomForestClassifier
0.89268130116156 0.8714926589504529 AdaBoostClassifier
0.6153618178241536 0.5742070855323529 GradientBoostingClassifier
0.6133200291570867 0.8008651589124296 GaussianNB
0.789192423223769 0.722890905818593 RidgeClassifierCV


Although the Random forest achieves a high F1-score, that might vary if we change the random value. This is because of the series of decision trees and net average of their Result. Hence I decided to go with stacking the best classifiers

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Class'],axis=1), y, test_size = 0.15, random_state = 42)
estimators = [
        ('ABC' ,AdaBoostClassifier()),
        ('RCC',RidgeClassifierCV()),
        ('DTC',DecisionTreeClassifier())
]

clf = StackingClassifier(
    estimators=estimators, 
    final_estimator=RandomForestClassifier()
)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
f1_score(y_true=y_test,y_pred=y_pred,average="macro") , roc_auc_score(y_true=y_test,y_score=y_pred,average="macro")

(0.9243373439106988, 0.8985783053734658)

The stacked model now gives us a higher F1 score as well as Area under the curve. 